# **Gradio Spaces**

In [ ]:
!git clone https://huggingface.co/spaces/Pipatpong/VCM_Demo

Cloning into 'VCM_Demo'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 54 (delta 30), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (54/54), 11.87 KiB | 934.00 KiB/s, done.


In [1]:
!pip install -q gradio
!pip install -q transformers
!pip install -q accelerate
!pip install -q bitsandbytes
!pip install -q huggingface_hub
!pip install -q torch

In [20]:
import gradio as gr
import re
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "Pipatpong/vcm_santa"
device = "cuda" # for GPU usage or "cpu" for CPU usage
tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True, device_map="auto", load_in_8bit=True)

def generate(text, max_length, num_return_sequences=1):
    inputs = tokenizer.encode(text, padding=False, add_special_tokens=False, return_tensors="pt")
    outputs = model.generate(inputs, max_length=max_length, num_return_sequences=num_return_sequences)
    gen_text = "Assignment : " + tokenizer.decode(outputs[0])
    if gen_text.count("#") > 2:
      split_text = gen_text.split("#", 2)
      return split_text[0] + "#" + split_text[1]
    else:
      return gen_text

def extract_functions(text):
    function_pattern = r'def\s+(\w+)\((.*?)\):([\s\S]*?)return\s+(.*?)\n'
    functions = re.findall(function_pattern, text, flags=re.MULTILINE)
    extracted_text = []

    for function in functions:
        function_name = function[0]
        parameters = function[1]
        function_body = function[2]
        return_statement = function[3]

        extracted_function = f"def {function_name}({parameters}):\n    # Code Here\n    return {return_statement}\n"
        extracted_text.append(extracted_function)

    return extracted_text

def assignment(text, max_length):
    extracted_functions = extract_functions(generate(text, max_length))
    for function in extracted_functions:
        return function

demo = gr.Blocks()

with demo:
    with gr.Row():
      with gr.Column():
        inputs=[gr.inputs.Textbox(placeholder="Type here and click the button for the desired action.", label="Prompt"),
                gr.Slider(30, 150, step=10, label="Max_length"),
              ]
      outputs=gr.outputs.Textbox(label="Generated Text")

    with gr.Row():
      b1 = gr.Button("Assignment")
      b2 = gr.Button("Example Code")

      b1.click(assignment, inputs, outputs)
      b2.click(generate, inputs, outputs)

    examples = [
    ["generate a python for sum number"],
    ["generate a python function to find max min element of list"],
    ["generate a python function to find minimum of two numbers with test case"],
    ]

    gr.Examples(examples=examples, inputs=inputs, cache_examples=False)

demo.launch(share=False, debug=False)

You are loading your model in 8bit or 4bit but no linear modules were found in your model. this can happen for some architectures such as gpt2 that uses Conv1D instead of Linear layers. Please double check your model architecture, or submit an issue on github if you think this is a bug.
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>